<a href="https://colab.research.google.com/github/ywan3223/ECE9063/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [370]:
# importing the libraries needed
import numpy as np # linear algebra
import pandas as pd # data processing aka love of my life
import matplotlib.pyplot as plt # plotting cute graphs
%matplotlib inline

import seaborn as sns
plt.style.use('seaborn-whitegrid')
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from datetime import datetime
import math
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.metrics import accuracy_score

In [371]:
# read the files
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')
store = pd.read_csv('../stores.csv')
feature = pd.read_csv('../features.csv')

### Clean up data

In [372]:
# merge the data for train dataset
df_train_merge = train.merge(store, how='left', on='Store')
df_train = df_train_merge.merge(feature, how='left', on=('Store', 'Date', 'IsHoliday'))
df_train = df_train[df_train['Store'] <= 1]

In [373]:
# merge the data for test dataset
df_test_merge = test.merge(store, how='left', on='Store')
df_test = df_test_merge.merge(feature, how='left', on=('Store', 'Date', 'IsHoliday'))
df_test = df_test[df_test['Store'] <= 1]

In [374]:
# split date for train dataset
df_train = pd.get_dummies(df_train, columns=["Type"])
df_train['Month'] = pd.to_datetime(df_train['Date']).dt.month
df_train = df_train.drop(columns=["Date"])

In [375]:
# split date for test dataset
df_test = pd.get_dummies(df_test, columns=["Type"])
df_test['Month'] = pd.to_datetime(df_test['Date']).dt.month
df_test = df_test.drop(columns=["Date"])

In [376]:
#replace Temperature, fule price and unemployment with averages to make data more reasonable                                              
df_train[["Temperature"]] = df_train[["Temperature"]].fillna(df_train[["Temperature"]].mean())
df_train[["Fuel_Price"]] = df_train[["Fuel_Price"]].fillna(df_train[["Fuel_Price"]].mean())
df_train[["Unemployment"]] = df_train[["Unemployment"]].fillna(df_train[["Unemployment"]].mean())

df_test[["Temperature"]] = df_test[["Temperature"]].fillna(df_test[["Temperature"]].mean())
df_test[["Fuel_Price"]] = df_test[["Fuel_Price"]].fillna(df_test[["Fuel_Price"]].mean())
df_test[["Unemployment"]] = df_test[["Unemployment"]].fillna(df_test[["Unemployment"]].mean())

In [377]:
# Replace the NaN values with 0
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [378]:
# drop the Extreme values
df_train = df_train[df_train['MarkDown1'] >0]

In [379]:
# diplsy train dataset after clean up
df_train

,Store,Dept,Weekly_Sales,IsHoliday,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type_A,Month
92,1,1,18689.54,False,151315,59.11,3.297,10382.90,6115.67,215.07,2406.62,6551.42,217.998085,7.866,1,11
93,1,1,19050.66,False,151315,62.25,3.308,6074.12,254.39,51.98,427.39,5988.57,218.220509,7.866,1,11
94,1,1,20911.25,True,151315,60.14,3.236,410.31,98.00,55805.51,8.00,554.92,218.467621,7.866,1,11
95,1,1,25293.49,False,151315,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,218.714733,7.866,1,12
96,1,1,33305.92,False,151315,43.93,3.158,4640.65,19.00,105.02,3639.42,14461.82,218.961846,7.866,1,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10239,1,99,20.06,False,151315,80.49,3.638,21442.73,102.32,21.70,13209.64,3032.96,222.305480,6.908,1,8
10240,1,99,0.05,True,151315,83.96,3.730,5204.68,35.74,50.94,4120.32,2737.17,222.439015,6.908,1,9
10241,1,99,0.03,False,151315,74.97,3.717,17212.52,7.00,18.79,1523.11,7992.72,222.582019,6.908,1,9
10242,1,99,635.00,False,151315,68.55,3.617,8077.89,0.00,18.22,3617.43,3626.14,223.181477,6.573,1,10


In [380]:
# display test dataset after clean up
df_test

,Store,Dept,IsHoliday,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type_A,Month
0,1,1,False,151315,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573000,1,11
1,1,1,False,151315,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573000,1,11
2,1,1,False,151315,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573000,1,11
3,1,1,True,151315,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573000,1,11
4,1,1,False,151315,52.34,3.207,2460.03,0.00,3838.35,150.57,6966.34,223.610984,6.573000,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2778,1,99,False,151315,76.44,3.439,3514.75,742.59,116.15,1015.44,5137.09,0.000000,6.509312,1,6
2779,1,99,False,151315,79.86,3.431,12592.32,527.48,402.88,7474.56,2484.44,0.000000,6.509312,1,6
2780,1,99,False,151315,79.85,3.422,7649.99,3503.29,1766.77,9454.96,1079.89,0.000000,6.509312,1,7
2781,1,99,False,151315,79.26,3.556,3117.04,1060.39,199.05,1012.30,5381.72,0.000000,6.509312,1,7


### set up train value

In [381]:
from sklearn.model_selection import train_test_split

In [382]:
# DF x & y

train_X = df_train.drop(["Weekly_Sales"], axis=1)
train_y = df_train["Weekly_Sales"]
test_X = df_test

X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.2, shuffle=False, stratify=None)

### algorithms

In [383]:
# create model linear regression
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [384]:
# error
print("Error for linear regression")
print("MAE: ", metrics.mean_absolute_error(y_test, y_pred)) 
print("MSE: ", metrics.mean_squared_error(y_test, y_pred)) 
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 


Error for linear regression
MAE:  46859.845032294004
MSE:  3894424636.105531
RMSE:  62405.325382578776


In [385]:
# create model svm
svm = svm.SVR()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

In [386]:
# error
print("Error for support vector machine")
print("MAE: ", metrics.mean_absolute_error(y_test, y_pred)) 
print("MSE: ", metrics.mean_squared_error(y_test, y_pred)) 
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


Error for support vector machine
MAE:  46326.78357465562
MSE:  3830649686.910589
RMSE:  61892.24254226526


In [387]:
# create model randomforest
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [388]:
# error
print("Error for support Random Forest")
print("MAE: ", metrics.mean_absolute_error(y_test, y_pred))
print("MSE: ", metrics.mean_squared_error(y_test, y_pred)) 
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

Error for support Random Forest
MAE:  36096.03602172131
MSE:  2378215450.7445602
RMSE:  48766.9503941405


### Best algorithm 

Compared MAE of different algorithm, Randomforest is the best model.


### Predict value

In [334]:
predicted_test = rf.predict(df_test[['Store','Size','Dept','Month','IsHoliday','Temperature','Fuel_Price','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','Type_A','CPI','Unemployment']])
df_test['weeklySales'] = predicted_test
df_test['id'] = df_test['Store'].astype(str) + '_' +  df_test['Dept'].astype(str) + '_' +  test['Date'].astype(str)
df_test_pre = df_test[['id', 'weeklySales']]
df_test_pre = df_test_pre.rename(columns={'id': 'Id', 'weeklySales': 'Weekly_Sales'})
df_test_pre.head()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


,Id,Weekly_Sales
0,1_1_2012-11-02,18405.3186
1,1_1_2012-11-09,18326.7742
2,1_1_2012-11-16,17922.5448
3,1_1_2012-11-23,18293.3262
4,1_1_2012-11-30,18331.6068
